In [142]:
import numpy as np

In [184]:
train = open('../data/processed/train.word','r').readlines()
freqsf = open('../data/processed/freqs.txt', 'r').readlines()

In [185]:
s1s = [l.split('\t')[1].split(' ') for l in train]
s2s = [l.split('\t')[2].split(' ') for l in train]
labels = [int(l.split('\t')[3]) for l in train]
words = [s.split(' ')[0] for s in freqsf if s[0] != ' ']
freqs = [int(s.strip().split(' ')[1]) for s in freqsf  if s[0] != ' ']

In [186]:
count = [0,0]
for l in labels:
    count[l] += 1
print(count, float(count[0])/count[1])
ratio = float(count[0])/count[1]

[27676, 7735] 3.578021978021978


In [202]:
def analyze_boolean_feature(func, verbose=False):
    ratio = float(count[0])/count[1]
    eps = 1e-6
#     ratio = 1.
    n = 0
    results = [[0, 0], [0, 0]]
    hit = [0,0]
    cls = [0,0]
    for s1, s2, l in zip(s1s, s2s, labels):
        n += 1
        r = func(s1, s2, l)
        results[r][l] += 1
        hit[r] += 1
        cls[l] += 1
    PCF = np.zeros((2,2)) # P(C|F)
    PFC = np.zeros((2,2)) # P(F|C)
    for c in [0,1]:
        for f in [0,1]: 
            PCF[c,f] = results[f][c]/float(hit[f] + eps) 
    for c in [0,1]:
        for f in [0,1]: 
            PFC[f,c] = results[f][c]/float(cls[c] + eps) 
    if verbose:
        print("# C            0         1")
        print("# P(C|F=0)   {0:.3f}      {1:.3f}".format(PCF[0,0], PCF[1,0]))
        print("# P(C|F=1)   {0:.3f}      {1:.3f}".format(PCF[0,1], PCF[1,1]))
#         print("# Feature=0  {0:.3f}      {1:.3f}".format(results[0][0]/float(n), results[0][1]/float(n)*ratio))
#         print("# Feature=1  {0:.3f}      {1:.3f}".format(results[1][0]/float(n), results[1][1]/float(n)*ratio))  
        print("# P(F=0|C)   {0:.3f}      {1:.3f}".format(PFC[0,0], PFC[0,1]))
        print("# P(F=1|C)   {0:.3f}      {1:.3f}".format(PFC[1,0], PFC[1,1]))
    return PCF, PFC

In [201]:
def func(s1, s2, l):
    # 只在一个句子中有该词
    word = u'滴滴' # 要注意同义词, 手动设置近义疑问词
    if (word in s1 and word not in s2) or (word in s2 and word not in s1):
#         if l == 1:
#             print(s1, s2)
        return 1
    else:
        return 0

analyze_boolean_feature(func, True)
# 花呗
#              0         1
# Feature=0  0.787      0.815
# Feature=1  0.931      0.265
# Feature=0  0.756      0.782
# Feature=1  0.038      0.011

# 借呗
#              0         1
# Feature=0  0.786      0.822
# Feature=1  0.946      0.208
# Feature=0  0.749      0.783
# Feature=1  0.044      0.010

# C            0         1
# P(C|F=0)   0.781      0.219
# P(C|F=1)   0.882      0.118
# P(F=0|C)   0.998      0.999
# P(F=1|C)   0.002      0.001


array([[ 0.78137113,  0.88235293],
       [ 0.21862887,  0.11764706]])

In [190]:
def build_func(word):
    def func(s1, s2, l):
        # 只在一个句子中有该词
        if (word in s1 and word not in s2) or (word in s2 and word not in s1):
            return 1
        else:
            return 0
    return func
    
for w, f in zip(words, freqs):
    if f > 100:
        func = build_func(w)
        PCF, PFC = analyze_boolean_feature(func)
        if np.abs(PCF[0,0] - PCF[0,1]) > 0.1:
            print("{0}\t P(C=0|F=0)={1:.3f}, P(C=0|F=1)={2:.3f}".format(w, PCF[0,0], PCF[0,1]))
        
    

花呗	 P(C=0|F=0)=0.776, P(C=0|F=1)=0.927
借呗	 P(C=0|F=0)=0.774, P(C=0|F=1)=0.956
分期	 P(C=0|F=0)=0.776, P(C=0|F=1)=0.891
多少	 P(C=0|F=0)=0.779, P(C=0|F=1)=0.881
逾期	 P(C=0|F=0)=0.779, P(C=0|F=1)=0.912
提前	 P(C=0|F=0)=0.779, P(C=0|F=1)=0.934
银行卡	 P(C=0|F=0)=0.780, P(C=0|F=1)=0.884
余额	 P(C=0|F=0)=0.779, P(C=0|F=1)=0.920
淘宝	 P(C=0|F=0)=0.780, P(C=0|F=1)=0.894
临时	 P(C=0|F=0)=0.780, P(C=0|F=1)=0.955
影响	 P(C=0|F=0)=0.780, P(C=0|F=1)=0.918
利息	 P(C=0|F=0)=0.781, P(C=0|F=1)=0.886
信用卡	 P(C=0|F=0)=0.780, P(C=0|F=1)=0.946
手续费	 P(C=0|F=0)=0.781, P(C=0|F=1)=0.886
扣	 P(C=0|F=0)=0.780, P(C=0|F=1)=0.881
是不是	 P(C=0|F=0)=0.779, P(C=0|F=1)=0.895
最低	 P(C=0|F=0)=0.780, P(C=0|F=1)=0.982
宝	 P(C=0|F=0)=0.781, P(C=0|F=1)=0.887
不是	 P(C=0|F=0)=0.780, P(C=0|F=1)=0.907
算	 P(C=0|F=0)=0.780, P(C=0|F=1)=0.910
身份证	 P(C=0|F=0)=0.781, P(C=0|F=1)=0.888
为啥	 P(C=0|F=0)=0.783, P(C=0|F=1)=0.676
积分	 P(C=0|F=0)=0.781, P(C=0|F=1)=0.885
电费	 P(C=0|F=0)=0.781, P(C=0|F=1)=0.929
充值	 P(C=0|F=0)=0.782, P(C=0|F=1)=0.662
密码	 P(C=0|F=0)=0.781, P

In [163]:
def contains(words, l):
    for w in words:
        if w in l:
            return True
    return False

def func(s1, s2, l):
    # 只在一个句子中有该类词
    word = [u'借呗'] # 要注意同义词, 手动设置近义疑问词
    if (contains(word, s1) and not contains(word, s2)) or (contains(word, s2) and not contains(word, s1)):
#         if l == 1:
#             print(s1, s2)
        return 1
    else:
        return 0

analyze_boolean_feature(func)

In [117]:
def func(s1, s2, l):
    # 两个句子里都有该词
    word = u'借呗'
    if word in s1 and word in s2:
        return 1
    else:
        return 0

analyze_boolean_feature(func)

# C            0         1
# Feature=0  0.804      0.750
# Feature=1  0.766      0.898
# Feature=0  0.571      0.532
# Feature=1  0.223      0.261
# P(F=0|C)   0.719      0.671
# P(F=1|C)   0.281      0.329


In [136]:
def func(s1, s2, l):
    # 两个句子里都没有该词
    word = u'花呗'
    if word not in s1 and word not in s2:
        return 1
    else:
        return 0

analyze_boolean_feature(func)

# C            0         1
# Feature=0  0.807      0.742
# Feature=1  0.760      0.919
# Feature=0  0.572      0.526
# Feature=1  0.221      0.267
# P(F=0|C)   0.722      0.663
# P(F=1|C)   0.278      0.337
